<a href="https://colab.research.google.com/github/iamJohnnySam/iamJohnnySam_SecuritySystem/blob/main/CCTV_Image_Classifier_2023_10_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [1]:
# Import Data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

In [3]:
rootLoc1 = '/content/drive/My Drive/Personal Projects/CCTV/A01/'
rootLoc2 = '/content/drive/My Drive/Personal Projects/CCTV/A02/'

# Split Images

In [4]:
import os
import random
import shutil

In [5]:
def split_photos_sub (dir, folder, cat, tr, va, te):
    train_folder = folder + "/train/" + cat
    test_folder = folder + "/test/predict"
    val_folder = folder + "/validation/" + cat

    for folder_path in [train_folder, val_folder, test_folder]:
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    random.seed(42)
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    imgs_list = [filename for filename in os.listdir(dir) if os.path.splitext(filename)[-1] in image_extensions]

    random.shuffle(imgs_list)

    print(len(imgs_list))
    train_size = int(len(imgs_list) * tr)
    val_size = int(len(imgs_list) * va)
    test_size = int(len(imgs_list) * te)

    for i, f in enumerate(imgs_list):
        if i < train_size:
            dest_folder = train_folder
        elif i < train_size + val_size:
            dest_folder = val_folder
        else:
            dest_folder = test_folder

        shutil.copy(os.path.join(dir, f), os.path.join(dest_folder, f))


def split_photos (dir, folder, tr, va, te):
    if not os.path.exists(folder):
        os.makedirs(folder)
        os.makedirs(folder + "/train")
        os.makedirs(folder + "/test")
        os.makedirs(folder + "/validation")

    split_photos_sub (dir + "0", folder, "0", tr, va, te)
    split_photos_sub (dir + "1", folder, "1", tr, va, te)

# Classifier


In [6]:
from keras.preprocessing.image import ImageDataGenerator

def classify (loc):
    datagen = ImageDataGenerator()

    train_it = datagen.flow_from_directory(loc + '/train/',
                                           class_mode='binary',
                                           target_size = (288, 352),
                                           batch_size=64)
    val_it = datagen.flow_from_directory(loc + '/validation/',
                                         class_mode='binary',
                                         target_size = (288, 352),
                                         batch_size=64)
    test_it = datagen.flow_from_directory(loc + '/test/',
                                          class_mode='binary',
                                          target_size = (288, 352),
                                          batch_size=64)


    return (train_it, val_it, test_it)

# Graph Generator

In [7]:
import matplotlib.pyplot as plt
def plotNNGraph (history):
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# Convert

In [8]:
import tensorflow as tf

In [9]:
def convert_model (model, cat):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    # Save the model.
    with open('/content/drive/My Drive/Personal Projects/CCTV/model' + cat + '.tflite', 'wb') as f:
        f.write(tflite_model)

# Camera Channel A01

In [10]:
split_photos (rootLoc1, "A01", 0.7, 0.2, 0.1)
train, val, test = classify ("A01")

1681
1676
Found 2349 images belonging to 2 classes.
Found 671 images belonging to 2 classes.
Found 337 images belonging to 1 classes.


In [ ]:
cnn = Sequential()
cnn.add(Conv1D(64, (3),
               activation='relu',
               padding='same',
               kernel_initializer='random_normal',
               input_shape=(288, 352, 3)))

cnn.add(MaxPooling2D(pool_size = (2)))
cnn.add(Dropout(0.2))
cnn.add(Conv1D(64, kernel_size = (3), activation = 'relu',
               kernel_initializer='random_normal'))
cnn.add(MaxPooling2D(pool_size = (2)))
cnn.add(Conv1D(64, kernel_size = (3), activation = 'relu',
               kernel_initializer='random_normal'))
cnn.add(MaxPooling2D(pool_size = (2)))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(32, activation='relu'))
cnn.add(Dropout(0.2))
cnn.add(Dense(8, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

cnn.compile(loss="binary_crossentropy",
            optimizer="adam",
            metrics=['accuracy'])

history = cnn.fit(train,
                validation_data = val,
                steps_per_epoch = train.n//train.batch_size,
                validation_steps = val.n//val.batch_size,
                epochs=5)

plotNNGraph(history)
convert_model (cnn, "A01")

Epoch 1/5
36/36 [==============================] - 777s 22s/step - loss: 4.4029 - accuracy: 0.6376 - val_loss: 0.5321 - val_accuracy: 0.8547
Epoch 2/5
36/36 [==============================] - 771s 21s/step - loss: 0.5637 - accuracy: 0.8162 - val_loss: 0.5325 - val_accuracy: 0.8484
Epoch 3/5
19/36 [==============>...............] - ETA: 5:16 - loss: 0.5079 - accuracy: 0.8354

# Camera Channel A02

In [ ]:
split_photos (rootLoc1, "A02", 0.7, 0.2, 0.1)
train, val, test = classify ("A02")

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(64, (3),
               activation='relu',
               padding='same',
               kernel_initializer='random_normal',
               input_shape=(288, 352, 3)))

cnn.add(MaxPooling2D(pool_size = (2)))
cnn.add(Dropout(0.2))
cnn.add(Conv1D(64, kernel_size = (3), activation = 'relu',
               kernel_initializer='random_normal'))
cnn.add(MaxPooling2D(pool_size = (2)))
cnn.add(Conv1D(64, kernel_size = (3), activation = 'relu',
               kernel_initializer='random_normal'))
cnn.add(MaxPooling2D(pool_size = (2)))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(32, activation='relu'))
cnn.add(Dropout(0.2))
cnn.add(Dense(8, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

cnn.compile(loss="binary_crossentropy",
            optimizer="adam",
            metrics=['accuracy'])

history = cnn.fit(train,
                validation_data = val,
                steps_per_epoch = train.n//train.batch_size,
                validation_steps = val.n//val.batch_size,
                epochs=5)

plotNNGraph(history)
convert_model (cnn, "A02")